In [51]:
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
import warnings
warnings.filterwarnings("ignore")

In [2]:
returns = pd.read_pickle("../Data/returns.pkl")
returns

,905270,921795,904261,905261,916328,923024,936365,902355,912215,929813,...,9660J1,69568X,543755,77463M,29235J,131745,69487D,68157P,9110RA,292703
date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,-0.835165,-0.475059,5.000000,-0.810537,0.361421,0.000000,-3.030303,0.000000,-0.431034,-1.315789,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,-0.797872,-1.272872,2.380952,-1.634321,-0.180060,3.454545,-1.562500,0.530612,0.225108,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,-2.546917,1.531023,-0.372093,-1.661475,-1.079730,1.054482,-3.174603,0.487211,-0.656531,-1.333333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,2.017423,-0.873016,0.000000,2.534319,0.000000,0.000000,0.000000,1.010101,0.434783,3.405405,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-24,NaN,NaN,NaN,NaN,0.136600,NaN,0.349877,NaN,NaN,NaN,...,0.376702,4.400221,1.590198,-0.991004,1.682900,3.364703,2.271817,0.634962,4.859794,1.204016
2021-05-25,NaN,NaN,NaN,NaN,0.946372,NaN,0.542355,NaN,NaN,NaN,...,0.230947,-0.288569,0.449063,-1.661106,0.439588,-0.548765,0.185928,0.727530,1.884279,-0.498203
2021-05-26,NaN,NaN,NaN,NaN,-1.393581,NaN,0.616491,NaN,NaN,NaN,...,1.296083,2.388001,-0.166049,3.255613,2.317760,4.702784,0.673960,2.336242,-0.429666,-1.520393


In [3]:
returns = returns.iloc[1:]

In [4]:
# dropping columns with all NaN
drop_columns = []

for col in returns.columns:
    if returns[col].isnull().all() == True:
        drop_columns.append(col)
        
returns.drop(columns=drop_columns, inplace=True)

In [8]:
def get_investable(t, n_rows):
    "Find stocks in investable universe at time t\
    (stocks in the S&P500 that have prices recorded for the last n_rows days)"
    
    df_investable = returns.copy(deep = True).sort_index(ascending = False)
    
    most_recent = t
    least_recent = t + pd.DateOffset(-n_rows)
    
    #take n_rows worth of data upto time specified
    df_investable = df_investable[most_recent+pd.DateOffset(-1):least_recent]
    
    #find all stocks that exist in the S&P at this time period
    investable_universe = []
    for col in df_investable.columns:
        if ~df_investable[col].iloc[:n_rows].isna().any():
            investable_universe.append(col)
        
    df_investable = df_investable[investable_universe]
    
    return df_investable

In [60]:
t = pd.to_datetime("2020-02-25")

get_investable(t, 80)

,916328,936365,905271,905113,905802,905425,906156,916305,992816,921093,...,311917,69568X,543755,77463M,29235J,131745,69487D,68157P,9110RA,292703
date,,,,,,,,,,,,,,,,,,,,,
2020-02-24,-3.224700,-7.807808,-3.702396,-1.195098,-6.519714,-1.641923,-4.974055,-5.967078,-4.750040,-2.460137,...,-2.020070,-7.459489,-2.467504,-6.481638,-6.091830,-4.945782,-4.950192,-2.934848,-7.530828,-3.530293
2020-02-21,-1.141759,-6.966998,-0.168363,-0.682511,-0.463535,-0.654425,-1.230048,-2.409639,-2.263503,-0.746100,...,-1.364204,0.176838,-1.604162,0.595846,-4.036727,-0.943149,-3.189911,-0.666838,-1.943285,-1.020350
2020-02-20,-1.007162,-2.767402,-0.164178,-1.085978,1.809029,-1.082126,-0.255605,0.789314,-1.025895,0.591312,...,-0.471153,-1.963115,0.348053,2.656414,-1.813720,2.884097,-0.111152,0.610700,0.028798,-1.631715
2020-02-19,0.540054,3.533134,0.031282,2.682395,3.974485,-0.795553,0.980826,0.816160,1.448276,-1.079865,...,-1.072034,6.875586,0.767207,42.444610,1.149301,0.487541,0.806753,4.644464,0.915432,1.375625
2020-02-18,-0.869953,2.856626,-0.105465,-0.539684,1.325820,0.549345,-0.198719,1.093234,-1.831051,-0.714764,...,1.049026,7.295914,0.573192,-2.238988,-3.065014,0.135612,-2.341698,-1.907116,-0.159582,-0.317316
2020-02-14,1.105097,1.430405,0.443346,1.244447,0.583431,0.884784,1.048639,-4.810524,0.024625,-1.104484,...,2.371351,-0.493781,1.024499,-0.604741,-0.125372,-3.683386,-1.160862,-2.151188,1.352742,2.426001
2020-02-13,-0.123888,1.187604,0.306966,0.426659,2.075889,1.041360,0.915641,-6.171702,-0.712103,-0.745451,...,0.422572,4.784371,1.790977,-0.385542,0.367011,2.737520,0.623957,1.785086,2.409276,0.566159
2020-02-12,-0.426152,0.167286,0.669546,2.467058,4.647436,-0.420661,0.459926,-0.348816,2.374769,-1.105811,...,-0.920232,-0.915507,0.730760,4.639435,0.638455,1.236245,1.704545,3.166107,1.389313,-1.148165
2020-02-11,0.985277,2.946804,0.278098,1.169012,2.202002,0.294349,0.294918,0.609531,-0.603328,0.963222,...,0.764667,0.401864,-0.091262,2.719503,2.893751,1.363261,1.749381,0.850869,2.088529,9.877013


In [100]:
# Converts investable dataframe into 3D tensor for input into RNN fitting
def formatX(investable_df):
    train_values = []
    for i in range(len(investable_df.index)):
        train_values.append(investable_df.iloc[i].values)
    train_values = np.array(train_values)
    train_values = np.reshape(train_values, (train_values.shape[0], 1, train_values.shape[1]))
    return train_values

In [101]:
formatX(get_investable(t, 80))

array([[[-3.22469983e+00, -7.80780781e+00, -3.70239636e+00, ...,
         -2.93484809e+00, -7.53082795e+00, -3.53029256e+00]],

       [[-1.14175899e+00, -6.96699843e+00, -1.68363352e-01, ...,
         -6.66837651e-01, -1.94328487e+00, -1.02035000e+00]],

       [[-1.00716204e+00, -2.76740238e+00, -1.64177938e-01, ...,
          6.10700155e-01,  2.87976962e-02, -1.63171470e+00]],

       ...,

       [[ 4.13907285e-01,  7.60649087e-02,  0.00000000e+00, ...,
         -9.65078102e-01,  1.70616114e-01,  6.83854202e-03]],

       [[ 8.22701800e-01,  1.31004367e+00, -5.71874188e-01, ...,
          1.46375934e+00, -3.21239607e-01,  1.05037662e+00]],

       [[-1.88348152e+00, -1.76633863e+00, -5.21484291e-01, ...,
         -2.63416552e+00, -1.18569695e+00, -1.03945839e+00]]])

In [102]:
# Formats output for a stock as a np array

def formatY(stock, t, n_rows):
    most_recent = t
    least_recent = t + pd.DateOffset(-n_rows)
    return returns[stock].loc[least_recent:most_recent].shift(-1).iloc[:-1].values[::-1]

In [106]:
# Creating a Simple RNN for an inputted stock

def get_model(stock, t, n_rows, investable_df):
    convertedX = convertToMatrix(investable_df)
    convertedY = formatY(stock, t, n_rows)
    
    model = Sequential()
    model.add(SimpleRNN(units=32, input_shape=(convertedX.shape[1], convertedX.shape[2]), activation="relu"))
    model.add(Dense(8, activation="relu")) 
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='rmsprop')
    
    model.fit(convertedX, convertedY, epochs=100, batch_size=16, verbose=2)
    
    return model

In [132]:
# Predicts stock price at a given date
import datetime

look_back = 80

def pred_date(stock, date):
    investable = get_investable(date, look_back)
    if stock not in investable.columns:
        print(stock + " is not in the investable universe")
        return
    if investable.shape[0] == 0:
        print("Investable universe is empty")
        return
    convertedX = formatX(investable.iloc[0:1])
    model = get_model(stock, date - datetime.timedelta(days=1), look_back, investable)
    return model.predict(convertedX)

In [133]:
pred_date("916328", t)

Epoch 1/100
4/4 - 1s - loss: 1.6840
Epoch 2/100
4/4 - 0s - loss: 0.6417
Epoch 3/100
4/4 - 0s - loss: 0.3371
Epoch 4/100
4/4 - 0s - loss: 0.3812
Epoch 5/100
4/4 - 0s - loss: 0.3912
Epoch 6/100
4/4 - 0s - loss: 1.0992
Epoch 7/100
4/4 - 0s - loss: 0.3982
Epoch 8/100
4/4 - 0s - loss: 0.1761
Epoch 9/100
4/4 - 0s - loss: 0.1473
Epoch 10/100
4/4 - 0s - loss: 0.1494
Epoch 11/100
4/4 - 0s - loss: 0.1484
Epoch 12/100
4/4 - 0s - loss: 0.1961
Epoch 13/100
4/4 - 0s - loss: 0.1008
Epoch 14/100
4/4 - 0s - loss: 0.1750
Epoch 15/100
4/4 - 0s - loss: 0.2110
Epoch 16/100
4/4 - 0s - loss: 0.1927
Epoch 17/100
4/4 - 0s - loss: 0.2014
Epoch 18/100
4/4 - 0s - loss: 0.1291
Epoch 19/100
4/4 - 0s - loss: 0.0896
Epoch 20/100
4/4 - 0s - loss: 0.0788
Epoch 21/100
4/4 - 0s - loss: 0.0784
Epoch 22/100
4/4 - 0s - loss: 0.1115
Epoch 23/100
4/4 - 0s - loss: 0.1444
Epoch 24/100
4/4 - 0s - loss: 0.2237
Epoch 25/100
4/4 - 0s - loss: 0.1554
Epoch 26/100
4/4 - 0s - loss: 0.1773
Epoch 27/100
4/4 - 0s - loss: 0.1515
Epoch 28/1

array([[-3.554593]], dtype=float32)

In [134]:
returns.loc[t, "916328"]

-4.135649296939626

In [105]:
convertedX = convertToMatrix(get_investable(t, 80))
model.predict(convertedX)

array([[-4.149425  ],
       [-3.1876523 ],
       [-1.0897539 ],
       [-0.8734021 ],
       [ 0.5606952 ],
       [-0.91085577],
       [ 1.1199747 ],
       [ 0.04846259],
       [-0.4195336 ],
       [ 1.0298836 ],
       [ 0.4942839 ],
       [-1.686663  ],
       [ 0.11253934],
       [ 1.0739907 ],
       [ 1.544517  ],
       [-0.01053615],
       [-2.222149  ],
       [-0.01053615],
       [-0.15079392],
       [-0.01053615],
       [-0.01053615],
       [-0.36814272],
       [-1.2524929 ],
       [ 2.3143144 ],
       [ 0.99493796],
       [ 1.8462325 ],
       [ 1.0259273 ],
       [ 1.9200814 ],
       [ 0.4288095 ],
       [-0.27803186],
       [-1.1819147 ],
       [ 1.0238563 ],
       [ 0.7016907 ],
       [-0.54143924],
       [ 0.48038727],
       [-1.3009133 ],
       [ 0.14288984],
       [-0.01053615],
       [-0.6819617 ],
       [ 0.15293474],
       [ 0.02162542],
       [ 0.1357847 ],
       [ 0.9609144 ],
       [-0.74740994],
       [ 0.4720417 ],
       [-0